<a href="https://colab.research.google.com/github/quang-m-nguyen/DeepPGD/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.callbacks import Callback

# from keras_self_attention import SeqSelfAttention
# from keras.utils import to_categorical
from keras.layers import (
    Bidirectional,
    Conv1D,
    Dense,
    Dropout,
    Embedding,
    Flatten,
    Input,
    LayerNormalization,
    concatenate,
    LSTM
)
from keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from tensorflow.keras import initializers, layers
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
# global x_test, y_test, x_train, y_train
# global best_acc
# global accuracy_best_list

# # make all of these variable global
# # global x_test, y_test, x_train, y_train
# # global best_acc
# # global accuracy_best_list



In [5]:
def prepare_training_data():
    train_filename = '/content/drive/MyDrive/deepPGD/4mC/4mC_C.equisetifolia/train.tsv'
    test_filename = '/content/drive/MyDrive/deepPGD/4mC/4mC_F.vesca/test.tsv'

    x_test = np.array([])
    y_test = np.array([])
    x_train = np.array([])
    y_train = np.array([])

    test_labels = []
    three_er_list = []

    K_MER = 3


    train_data = pd.read_csv(train_filename,header = None, sep = "\t")
    test_data  = pd.read_csv(test_filename,header = None, sep = "\t")

    pro_x_train = train_data[2][1:]
    y_train =  train_data[1][:]
    pos_train_len = len(pro_x_train)

    pro_x_test =  test_data[2][1:]
    y_test = test_data[1][:]

    pro_x_data  = pd.concat([pro_x_train,pro_x_test],ignore_index= True )
    pro_y_data  = pd.concat([y_train,y_test],ignore_index= True )


    for i in range(1, len(pro_y_data)):
      if(pro_y_data[i] == "1"):
          test_labels.append([1,0])
      elif(pro_y_data[i] == "0"):
          test_labels.append([0,1])
      else:
          continue

    # K-mer Encoding for DNA Sequences
    #
    # Purpose:
    # - Transform variable-length DNA sequences into fixed-length feature representations
    # - Capture local sequence patterns that may be relevant to DNA methylation sites
    # - Create a suitable input format for machine learning models
    #
    # Functionality:
    # 1. Set k-mer size (K=3 in this case)
    # 2. For each DNA sequence:
    #    a. Convert to string and remove any extra characters
    #    b. Generate all possible k-mers (substrings of length K)
    #    c. Store k-mers for each sequence in a list
    # 3. Collect k-mer lists for all sequences in str_array
    #
    # Benefits:
    # - Captures local sequence context
    # - Provides fixed-length representation for variable-length sequences
    # - Reduces sequence complexity while retaining important features
    # - Facilitates efficient sequence comparison and analysis
    # - Improves feature extraction for machine learning models
    #
    # Example:
    # Input DNA sequence: "ATCGATCG"
    # Resulting k-mers (K=3): ["ATC", "TCG", "CGA", "GAT", "ATC", "TCG"]
    #
    # Data structure:
    # str_array = [
    #     ["ATC", "TCG", "CGA", "GAT", "ATC", "TCG"],
    # ]

    for i in pro_x_data:
        seq_str = str(i)
        seq_str = seq_str.strip('[]\'')
        t=0
        l=[]
        for index in range(len(seq_str)):
            t=seq_str[index:index+K_MER]
            if (len(t))==K_MER:
                l.append(t)
        three_er_list.append(l)



    # DNA Sequence Preprocessing
    # Purpose: Turn DNA chunks into number lists for machine learning
    # Steps:
    # 1. Assign a unique number to each DNA chunk (up to 30,000 most common chunks)
    # 2. Convert each DNA sequence to a list of these numbers tokens
    # 3. Make all lists the same length (48) by adding zeros at the end if needed

    # Example:
    # Input DNA sequences:
    #   ["ATCG", "CGTA", "ATCGATCG"]
    #
    # After assigning numbers:
    #   ATCG -> 1, CGTA -> 2, GATC -> 3
    #
    # Converted to number lists:
    #   [1, 2]
    #   [2, 1]
    #   [1, 3, 1]
    #
    # Final output (padded to length 48):
    #   [1, 2, 0, 0, ..., 0]  (46 zeros)
    #   [2, 1, 0, 0, ..., 0]  (46 zeros)
    #   [1, 3, 1, 0, ..., 0]  (45 zeros)

    tokenizer = Tokenizer(num_words = 30000)
    tokenizer.fit_on_texts(three_er_list)
    sequences = tokenizer.texts_to_sequences(three_er_list)
    sequences = pad_sequences(sequences, maxlen = 48, padding = "post")
    sequences = np.array(sequences)


    x_train,x_test = sequences[:pos_train_len],sequences[pos_train_len:]
    # print(x_train)

    y_train,y_test = test_labels[:pos_train_len],test_labels[pos_train_len:]

    return x_train, x_test, y_train, y_test





In [6]:
def create_masked_data(x_train, mask_percentage):
    """
    Create a masked version of the input data.

    Args:
    x_train (numpy.ndarray): Input data to be masked.
    mask_percentage (float): Percentage of data to be masked, between 0 and 1.

    Returns:
    numpy.ndarray: Masked version of the input data.
    """
    # Validate mask_percentage range
    if mask_percentage < 0.0 or mask_percentage > 1.0:
        raise ValueError("mask_percentage must be between 0 and 1.")

    # Create a boolean mask: True with probability mask_percentage
    mask = np.random.random(x_train.shape) < mask_percentage

    # Apply the mask: keep original values where mask is False, set to 0 where mask is True
    masked_data = np.where(mask, 0.0, x_train)

    return masked_data

In [11]:
def make_model_architecture(masked_data):
    t = time.time()
    my_time = int(round(t * 1000)) % 2147483648
    print(my_time)
    np.random.seed(my_time)

    sequence_input = Input(shape=(48,))
    embedding_layer = Embedding(30000, 16, input_length=48)
    embedded_sequences = embedding_layer(sequence_input)


    stem = LayerNormalization(epsilon=1e-6)(embedded_sequences)
    stem = Dropout(0.2)(stem)

    MLP_1 = Dense(
        int(16 * 2.0),
        name="Dense_0",
        kernel_initializer=initializers.GlorotUniform(),
        bias_initializer=initializers.RandomNormal(stddev=1e-6),
    )(stem)
    Activation_1 = layers.Activation("gelu")(MLP_1)
    Activation_1 = LayerNormalization(epsilon=1e-6)(Activation_1)
    Dropout_2 = Dropout(0.2)(Activation_1)

    MLP_2 = Dense(
        int(16 * 1.0),
        name="Dense_1",
        kernel_initializer=initializers.GlorotUniform(),
        bias_initializer=initializers.RandomNormal(stddev=1e-6),
    )(Dropout_2)
    MLP_2 = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=16)(MLP_2, stem)
    stem = stem + MLP_2
    stem = Dropout(0.2)(stem)

    lstm = Bidirectional(LSTM(16, return_sequences=True))(stem)
    lstm = layers.Activation("gelu")(lstm)
    lstm = LayerNormalization(epsilon=1e-6)(lstm)
    lstm = Dropout(0.5)(lstm)

    lstm1 = Bidirectional(LSTM(16, return_sequences=True))(lstm)
    lstm1 = layers.Activation("gelu")(lstm1)
    lstm1 = LayerNormalization(epsilon=1e-6)(lstm1)
    lstm1 = Dropout(0.2)(lstm1)
    lstm1 = lstm1 + lstm

    lstm2 = Bidirectional(LSTM(16, return_sequences=True))(lstm1)
    lstm2 = layers.Activation("gelu")(lstm2)
    lstm2 = LayerNormalization(epsilon=1e-6)(lstm2)
    lstm2 = Dropout(0.2)(lstm2)

    lstm2 = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=16)(lstm2, lstm)
    lstm2 = lstm2 + lstm

    cnn1_24 = Conv1D(
        filters=32, kernel_size=9, strides=1, activation="gelu", padding="causal"
    )(stem)
    cnn1_16 = Conv1D(
        filters=64, kernel_size=8, strides=1, activation="gelu", padding="causal"
    )(stem)
    cnn1_10 = Conv1D(
        filters=32, kernel_size=5, strides=1, activation="gelu", padding="causal"
    )(stem)
    merge_fla_1 = concatenate([cnn1_24, cnn1_16, cnn1_10], axis=2)
    merge_fla_1 = LayerNormalization(epsilon=1e-6)(merge_fla_1)
    merge_fla_1 = Dropout(0.2)(merge_fla_1)

    cnn2_24 = Conv1D(
        filters=32, kernel_size=9, strides=1, activation="gelu", padding="causal"
    )(merge_fla_1)
    cnn2_16 = Conv1D(
        filters=64, kernel_size=8, strides=1, activation="gelu", padding="causal"
    )(merge_fla_1)
    cnn2_10 = Conv1D(
        filters=32, kernel_size=5, strides=1, activation="gelu", padding="causal"
    )(merge_fla_1)

    merge_fla_2 = concatenate([cnn2_24, cnn2_16, cnn2_10], axis=2)
    merge_fla_2 = merge_fla_2 + merge_fla_1
    merge_fla_2 = LayerNormalization(epsilon=1e-6)(merge_fla_2)
    merge_fla_2 = Dropout(0.2)(merge_fla_2)

    cnn3_24 = Conv1D(
        filters=32, kernel_size=9, strides=1, activation="gelu", padding="causal"
    )(merge_fla_2)
    cnn3_16 = Conv1D(
        filters=64, kernel_size=8, strides=1, activation="gelu", padding="causal"
    )(merge_fla_2)
    cnn3_10 = Conv1D(
        filters=32, kernel_size=5, strides=1, activation="gelu", padding="causal"
    )(merge_fla_2)

    merge_fla_3 = concatenate([cnn3_24, cnn3_16, cnn3_10], axis=2)
    merge_fla_3 = merge_fla_2 + merge_fla_3
    merge_fla_3 = LayerNormalization(epsilon=1e-6)(merge_fla_3)
    merge_fla_3 = Dropout(0.2)(merge_fla_3)

    cnn4_24 = Conv1D(
        filters=32, kernel_size=9, strides=1, activation="gelu", padding="causal"
    )(merge_fla_3)
    cnn4_16 = Conv1D(
        filters=64, kernel_size=8, strides=1, activation="gelu", padding="causal"
    )(merge_fla_3)
    cnn4_10 = Conv1D(
        filters=32, kernel_size=5, strides=1, activation="gelu", padding="causal"
    )(merge_fla_3)

    merge_fla_4 = concatenate([cnn4_24, cnn4_16, cnn4_10], axis=2)
    merge_fla_4 = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=16)(
        merge_fla_1, merge_fla_4
    )
    merge_fla_4 = merge_fla_1 + merge_fla_4
    merge_fla_4 = LayerNormalization(epsilon=1e-6)(merge_fla_4)
    merge_fla_4 = Dropout(0.2)(merge_fla_4)
    lstm = concatenate([merge_fla_4, lstm2])

    lstm = Flatten()(lstm)
    # merge_fla = Dropout(0.2)(lstm)

    merge = Dense(200, activation="sigmoid")(lstm)
    merge = Dropout(0.5)(merge)

    preds = Dense(2, activation="softmax")(merge)
    model = Model(sequence_input, preds)

    model.summary()

    model.compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )

    return model


In [12]:
# Usage
x_train, x_test, y_train, y_test = prepare_training_data()
# print(x_train)
mask_percentage = 0.15  # Mask 15% of the data
x_train = create_masked_data(x_train, mask_percentage)
model = make_model_architecture(x_train)




193258382


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 48)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 48, 16)         │        480,000 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, 48, 16)         │             32 │ embedding_1[0][0]      │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 48, 16)         │              0 │ layer_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Dense_0 (Dense)           │ (None, 48, 32)         │            544 │ dropout_4[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 48, 32)         │              0 │ Dense_0[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_3     │ (None, 48, 32)         │             64 │ activation_1[0][0]     │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)       │ (None, 48, 32)         │              0 │ layer_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Dense_1 (Dense)           │ (None, 48, 16)         │            528 │ dropout_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_1    │ (None, 48, 16)         │          4,304 │ Dense_1[0][0],         │
│ (MultiHeadAttention)      │                        │                │ dropout_4[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 48, 16)         │              0 │ dropout_4[0][0],       │
│                           │                        │                │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_7 (Dropout)       │ (None, 48, 16)         │              0 │ add_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 48, 32)         │          4,640 │ dropout_7[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 48, 64)         │          8,256 │ dropout_7[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_2 (Conv1D)         │ (None, 48, 32)         │          2,592 │ dropout_7[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 48, 128)        │              0 │ conv1d[0][0],          │
│                           │                        │                │ conv1d_1[0][0],        │
│                           │                        │                │ conv1d_2[0][0]         │
├──────────────────────

 Total params: 2,466,074 (9.41 MB)

 Trainable params: 2,466,074 (9.41 MB)

 Non-trainable params: 0 (0.00 B)